In [1]:
import os
import json

In [2]:
split_file = os.path.join("workspace", "nnUNet_preprocessed", "splits_final.json")
data_dir = "data"
imagesTr_dir = "workspace/train/imagesTr"
labelsTr_dir = "workspace/train/labelsTr"
labelsTs_true_dir = os.path.join("workspace/test/labelsTs", "true")
manual_labels = "panorama_labels/manual_labels"
automatic_labels = "panorama_labels/automatic_labels"
os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)
os.makedirs(labelsTs_true_dir, exist_ok=True)

In [3]:
with open(split_file, "r") as f:
    splits = json.load(f)

split0 = splits[0]
train_ids = split0.get("train", [])
val_ids = split0.get("val", [])

In [4]:
all_labels_full = [
    os.path.join(manual_labels, f) for f in os.listdir(manual_labels)
] + [os.path.join(automatic_labels, f) for f in os.listdir(automatic_labels)]

In [5]:
def get_images(data: str | list, ids: list) -> list:
    """Gets all the images/labels from the specified folder and needed ids."""
    if isinstance(data, str):
        listing = [os.path.join(data, f) for f in os.listdir(data)]
    else:
        listing = data

    if not listing:
        raise ValueError(f"The directory {data} is empty or does not exist.")

    prefixes = tuple(ids)
    images = [file for file in listing if file.split("/")[-1].startswith(prefixes)]

    if not images:
        raise ValueError(f"No images found in {data} for the specified fold.")
    return images

In [6]:
import shutil

In [7]:
def move_images_to_folder(images: list, folder: str):
    for image in images:
        image_name = os.path.basename(image)
        new_image_path = os.path.join(folder, image_name)
        if not os.path.exists(new_image_path):
            shutil.copy(image, new_image_path)

In [8]:
train_labels = get_images(all_labels_full, train_ids)

In [9]:
move_images_to_folder(train_labels, labelsTr_dir)

In [10]:
test_labels = get_images(all_labels_full, val_ids)

In [11]:
move_images_to_folder(test_labels, labelsTs_true_dir)

In [12]:
train_images = get_images(data_dir, train_ids)

In [13]:
move_images_to_folder(train_images, imagesTr_dir)